# FIKH Model: Startup Shear

## Protocol-Specific Context

This notebook demonstrates **FIKH startup shear** predictions where the fractional order $\alpha$ critically affects:

1. **Stress overshoot timing**: Lower $\alpha$ → broader overshoot, longer tail
2. **Approach to steady state**: Power-law decay $\sim t^{-\alpha}$ (not exponential)
3. **Structure breakdown dynamics**: Mittag-Leffler decay vs classical exponential

**Why this matters**: Startup transients reveal the **power-law memory** that distinguishes FIKH from classical IKH ($\alpha=1$). The overshoot shape and decay provide direct constraints on $\alpha$.

> **Key physics**: During startup, structure $\lambda$ breaks down via $D_t^{\alpha} \lambda = -\Gamma \lambda |\dot{\gamma}|$. The Caputo derivative produces Mittag-Leffler relaxation $E_{\alpha}(-(t/\tau)^{\alpha})$ which interpolates between stretched exponential (short time) and power-law tail (long time).

> **Handbook:** See [FIKH Startup Protocol](../../docs/source/models/fikh/fikh.rst#start-up-of-steady-shear) for theory and parameter interpretation.

## Learning Objectives

1. Generate synthetic startup shear data using FIKH calibrated parameters
2. Observe how $\alpha$ affects stress overshoot shape and decay
3. Fit FIKH to startup data and recover calibrated $\alpha$
4. Compare fractional vs classical ($\alpha=1$) overshoot dynamics
5. Validate Bayesian inference on transient protocol

## Prerequisites

- NB01: Flow curve (parameter calibration)
- Understanding of Maxwell overshoot in startup shear

**Estimated Time:** 3-5 minutes (fast), 12-18 minutes (full)

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import os
import sys
import time
import warnings

# Add examples/utils to path
# Robust path resolution for execution from any directory
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fikh import FIKH

_nb_dir = Path(__file__).parent if "__file__" in dir() else Path.cwd()
_utils_candidates = [_nb_dir / ".." / "utils", Path("examples/utils"), _nb_dir.parent / "utils"]
for _p in _utils_candidates:
    if (_p / "fikh_tutorial_utils.py").exists():
        sys.path.insert(0, str(_p.resolve()))
        break
from fikh_tutorial_utils import (
    compute_fit_quality,
    get_fikh_param_names,
    load_pnas_startup,
    plot_alpha_sweep,
    print_alpha_interpretation,
    print_convergence_summary,
    print_parameter_comparison,
    save_fikh_results,
)

# Shared plotting utilities
sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    display_arviz_diagnostics,
    plot_nlsq_fit,
    plot_posterior_predictive,
)

# FAST_MODE: controlled by environment variable
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

jax, jnp = safe_import_jax()
verify_float64()

print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")
print(f"FAST_MODE: {FAST_MODE}")

## 2. Theory: Startup Shear with Fractional Memory

During startup shear at constant rate $\dot{\gamma}$, the FIKH model predicts:

1. **Initial elastic response**: $\sigma \approx G \cdot \gamma$
2. **Stress overshoot**: Peak occurs when plastic flow begins
3. **Steady-state approach**: Governed by structure evolution

### Alpha Effect on Startup

The fractional order $\alpha$ affects:
- **Overshoot timing**: Lower $\alpha$ → later peak (slower structure breakdown)
- **Overshoot magnitude**: Modified by memory kernel
- **Approach to steady-state**: Power-law vs exponential convergence

## 3. Load Data

In [ ]:
# Load PNAS startup data at gamma_dot = 1.0 s^-1
GAMMA_DOT = 1.0
time_data, stress_data = load_pnas_startup(gamma_dot=GAMMA_DOT)

print(f"Data points: {len(time_data)}")
print(f"Time range: [{time_data.min():.4f}, {time_data.max():.2f}] s")
print(f"Stress range: [{stress_data.min():.2f}, {stress_data.max():.2f}] Pa")
print(f"Shear rate: {GAMMA_DOT} 1/s")

In [ ]:
# Plot raw data
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(time_data, stress_data, "ko", markersize=5, label="Data")
ax.set_xlabel("Time [s]", fontsize=12)
ax.set_ylabel("Stress [Pa]", fontsize=12)
ax.set_title(f"PNAS Startup Shear Data ($\\dot{{\\gamma}}$ = {GAMMA_DOT} s$^{{-1}}$)", fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

## 4. NLSQ Fitting

In [ ]:
# Create and fit FIKH model
model = FIKH(include_thermal=False, alpha_structure=0.7)

# Compute strain from time and shear rate
strain_data = GAMMA_DOT * time_data

t0 = time.time()
model.fit(time_data, stress_data, test_mode="startup", strain=strain_data, method='scipy')
t_nlsq = time.time() - t0

param_names = get_fikh_param_names(include_thermal=False)

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"\nFitted parameters:")
for name in param_names:
    val = model.parameters.get_value(name)
    print(f"  {name:15s} = {val:.4g}")

In [ ]:
# Compute fit quality and plot with uncertainty band
stress_pred = model.predict(time_data, test_mode="startup", strain=strain_data)
metrics = compute_fit_quality(stress_data, stress_pred)

print(f"\nFit Quality:")
print(f"  R^2:   {metrics['R2']:.6f}")
print(f"  RMSE:  {metrics['RMSE']:.4g} Pa")

fig, ax = plot_nlsq_fit(
    time_data, stress_data, model, test_mode="startup",
    param_names=param_names, log_scale=False,
    xlabel="Time [s]", ylabel="Stress [Pa]",
    title=f"FIKH Startup Fit (R$^2$ = {metrics['R2']:.5f})",
    x_pred=time_data,
    strain=strain_data,
)
display(fig)
plt.close(fig)

## 5. Alpha Exploration

In [ ]:
# Define fine time grid for alpha sweep
time_fine = np.linspace(time_data.min(), time_data.max(), 200)

# Alpha sweep for startup shear
alpha_values = [0.3, 0.5, 0.7, 0.9, 0.99]

fig = plot_alpha_sweep(
    model,
    protocol="startup",
    alpha_values=alpha_values,
    x_data=time_fine,
    gamma_dot=GAMMA_DOT,
    figsize=(14, 5),
)

# Add data to left panel
fig.axes[0].plot(time_data, stress_data, "ko", markersize=3, alpha=0.5, label="Data")
fig.axes[0].legend(fontsize=8, loc="best")

display(fig)
plt.close(fig)

In [ ]:
# Physical interpretation
fitted_alpha = model.parameters.get_value("alpha_structure")
print_alpha_interpretation(fitted_alpha)

## 6. Bayesian Inference

In [ ]:
# Bayesian inference with NLSQ warm-start
initial_values = {name: model.parameters.get_value(name) for name in param_names}

NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1
# NUM_WARMUP = 1000; NUM_SAMPLES = 2000; NUM_CHAINS = 4  # production

print(f"Running NUTS: {NUM_WARMUP} warmup + {NUM_SAMPLES} samples x {NUM_CHAINS} chain(s)")
t0 = time.time()
result = model.fit_bayesian(
    time_data,
    stress_data,
    test_mode="startup",
    strain=strain_data,
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    initial_values=initial_values,
    seed=42,
)
t_bayes = time.time() - t0
print(f"\nBayesian inference time: {t_bayes:.1f} s")

### Convergence Diagnostics

**Bayesian Diagnostic Interpretation:**

| Metric | Target | Interpretation |
|--------|--------|----------------|
| **R-hat** | < 1.01 | Chain convergence (< 1.05 acceptable) |
| **ESS** | > 400 | Effective sample size (> 100 minimum) |
| **Divergences** | < 1% | NUTS sampling quality |

Passing diagnostics ensure reliable parameter estimates and credible intervals.

In [ ]:
# Convergence diagnostics
all_pass = print_convergence_summary(result, param_names)

print("\n### Diagnostic Interpretation Table")
print("| Metric | Target | Interpretation |")
print("|--------|--------|----------------|")
print("| R-hat | < 1.01 | Chain convergence (< 1.05 acceptable) |")
print("| ESS | > 400 | Effective sample size (> 100 minimum) |")
print("| Divergences | < 1% | Sampling quality |")

In [ ]:
# ArviZ diagnostic plots (trace, pair, forest, energy, autocorr, rank)
display_arviz_diagnostics(result, param_names, fast_mode=FAST_MODE)

In [ ]:
# Posterior predictive with 95% credible intervals
fig, ax = plot_posterior_predictive(
    time_data, stress_data, model, result,
    test_mode="startup", param_names=param_names,
    log_scale=False,
    xlabel="Time [s]", ylabel="Stress [Pa]",
    title="FIKH Startup Posterior Predictive",
    x_pred=time_data,
    strain=strain_data,
)
display(fig)
plt.close(fig)

In [ ]:
# Parameter comparison
posterior = result.posterior_samples
print_parameter_comparison(model, posterior, param_names)

## 7. Save Results

In [ ]:
save_fikh_results(model, result, "fikh", "startup", param_names)
print("\nResults saved for downstream analysis.")

## Key Takeaways

1. **Startup shear is highly sensitive to $\alpha$**:
   - Lower $\alpha$ → broader overshoot, slower decay
   - Overshoot decay follows power-law $\sim t^{-\alpha}$ not exponential

2. **Transient data constrains $\alpha$ better than steady-state**
3. **Mittag-Leffler relaxation** distinguishes FIKH from classical IKH
4. **Bayesian inference recovers calibrated $\alpha$ with uncertainty**
5. **Residual analysis** confirms quality of fit to transient overshoot

---

## Further Reading

- **[FIKH Startup Protocol](../../docs/source/models/fikh/fikh.rst#start-up-of-steady-shear)**: Mathematical details of Mittag-Leffler decay
- **[Protocol Equations](../../docs/source/models/fikh/fikh.rst#protocol-equations)**: Analytical approximations for overshoot timing

### Key References

1. Jaishankar, A. & McKinley, G.H. (2014). "A fractional K-BKZ constitutive formulation for describing the nonlinear rheology of multiscale complex fluids." *J. Rheol.*, 58, 1751-1788.
2. Dimitriou, C.J. & McKinley, G.H. (2014). "A comprehensive constitutive law for waxy crude oil: a thixotropic yield stress fluid." *Soft Matter*, 10, 6619-6644.

### Next Steps

**Next**: NB03 (Stress Relaxation) — power-law tails in relaxation reveal $\alpha$ effect